# Results Notebook

In [1]:
import hwacctools.quantization.quant as q
from stim_lib.stimulus_gen import generate_qracc_inputs
import numpy as np
import plotly.express as px
import pandas as pd
df = pd.DataFrame

def melt_row_col_data(arr,name):
    arr_df = df(arr)
    arr_df = arr_df.melt(
        ignore_index=False,
        var_name='Column',
        value_name='ADC_OUT_AMS',
    )
    arr_df['Row'] = arr_df.index
    return arr_df

### Comparing AMS vs RTL simulations of QRAcc

In [6]:
ams = np.loadtxt('../tb/qracc/inputs/adc_out.txt')
rtl = np.loadtxt('../tb/qracc/inputs/adc_out_rtlsims.txt')

ams_df = df(ams)
rtl_df = df(rtl)

# ams_df['Row'] = ams_df.index
# rtl_df['Row'] = rtl_df.index

ams_df = ams_df.melt(
    ignore_index=False,
    var_name='Column',
    value_name='ADC_OUT_AMS',
)
rtl_df = rtl_df.melt(
    ignore_index=False,
    var_name='Column',
    value_name='ADC_OUT_RTL',
)
ams_df['Row'] = ams_df.index
rtl_df['Row'] = rtl_df.index
# Join the two dataframes
adc_df = pd.merge(ams_df, rtl_df)

fig = px.scatter(adc_df, x='ADC_OUT_AMS', y='ADC_OUT_RTL')
fig.show()
fig.write_image('../images/rtl_vs_ams.svg')

In [ ]:
df(ams)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
1,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
5,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
6,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
7,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
8,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,...,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0
9,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


### Save current dataFrame

In [4]:
adc_df.to_csv('tb_qracc_results.csv', index=False,)